In [54]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [55]:
# Dados
Tarefas = [i for i in range(6)]
proc_tempo = {0: 0, 1: 64, 2: 53, 3: 63, 4: 99, 5: 189}
entrega = {0: 0, 1: 100, 2: 70, 3: 150, 4: 601, 5: 118}
M = 10000

# c) Objetivo de minimizar o número de tarefas atrasadas

In [60]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.T = Set(initialize=Tarefas)
T = model.T

# Parâmetros
model.p = Param(T, initialize=proc_tempo, within=NonNegativeReals)
model.d = Param(T, initialize=entrega, within=NonNegativeReals)
p, d = model.p, model.d

# Variáveis de decisão
model.x = Var(T * T, within=Binary)
model.C = Var(T, within=NonNegativeReals)
model.t = Var(T, within=NonNegativeReals)
model.y = Var(T, within=Binary)
x, C, t, y = model.x, model.C, model.t, model.y

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = sum(y[i] for i in T if i > 0))

# Restrições
# Um sucessor por tarefa
model.r_sucessor = ConstraintList()
for i in T:
    model.r_sucessor.add(expr = sum(x[i, j] for j in T if j != i) == 1)
# Um antecessor por tarefa
model.r_antecessor = ConstraintList()
for j in T:
    model.r_antecessor.add(expr = sum(x[i, j] for i in T if i != j) == 1)
# Tempo de término
model.r_termino = ConstraintList()
for i in T:
    for j in T:
        if j != i and j > 0:
            model.r_termino.add(expr = C[j] >= C[i] + p[j] * x[i, j] + M * (x[i, j] - 1))
# Tempos de atraso positivos
model.r_atraso = ConstraintList()
for i in T:
    if i > 0:
        model.r_atraso.add(expr = t[i] >= C[i] - d[i])
# Binário de contagem de tarefas atrasadas
model.r_contagem_atraso = ConstraintList()
for i in T:
    model.r_contagem_atraso.add(expr = t[i] <= M * y[i])

# model.pprint()

In [61]:
# Resolução
solver = SolverFactory('cplex')
resultado = solver.solve(model, tee=False)

In [62]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    print('Item c) Objetivo de minimizar o número de tarefas atrasadas \n')
    print('Tempo de fluxo total: {:.0f} minutos.'.format(sum(value(C[i]) for i in T)))
    # print('Soma dos atrasos: {:.0f} minutos.'.format(sum(value(t[i]) for i in T)))
    print('Número de tarefas atrasadas: {:.0f} tarefas.'.format(value(model.obj)))
    print()
    for i in T:
        if i > 0:
            print('Atraso da tarefa {}: {:.2f}'.format(i, value(C[i] - d[i])))
            print('Conclusão da tarefa {}: {:.2f}'.format(i, value(C[i])))
            print('Entrega da tarefa {}: {:.2f}'.format(i, value(d[i])))
            print()
else:
    print(resultado.solver.termination_condition)

Item c) Objetivo de minimizar o número de tarefas atrasadas 

Tempo de fluxo total: 1256 minutos.
Número de tarefas atrasadas: 2 tarefas.

Atraso da tarefa 1: 368.00
Conclusão da tarefa 1: 468.00
Entrega da tarefa 1: 100.00

Atraso da tarefa 2: -17.00
Conclusão da tarefa 2: 53.00
Entrega da tarefa 2: 70.00

Atraso da tarefa 3: -34.00
Conclusão da tarefa 3: 116.00
Entrega da tarefa 3: 150.00

Atraso da tarefa 4: -386.00
Conclusão da tarefa 4: 215.00
Entrega da tarefa 4: 601.00

Atraso da tarefa 5: 286.00
Conclusão da tarefa 5: 404.00
Entrega da tarefa 5: 118.00

